In [1]:
import sys
import unittest
import functools

import numpy as np
from scipy.linalg import expm
from scipy.special import erf

from qiskit.providers.aer.backends import PulseSimulator

from qiskit.compiler import assemble
from qiskit.quantum_info import state_fidelity
from qiskit.pulse import (Schedule, Play, ShiftPhase, SetPhase, Delay, Acquire, SamplePulse,
                          DriveChannel, ControlChannel, AcquireChannel, MemorySlot)
from qiskit.providers.aer.pulse.de.DE_Methods import ScipyODE
from qiskit.providers.aer.pulse.de.DE_Options import DE_Options
from qiskit.providers.aer.pulse.system_models.pulse_system_model import PulseSystemModel
from qiskit.providers.aer.pulse.system_models.hamiltonian_model import HamiltonianModel
from qiskit.providers.models.backendconfiguration import UchannelLO

In [2]:
def _1Q_constant_sched(total_samples, amp=1.):
    """Creates a runnable schedule for 1Q with a constant drive pulse of a given length.

    Args:
        total_samples (int): length of pulse
        amp (float): amplitude of constant pulse (can be complex)

    Returns:
        schedule (pulse schedule): schedule with a drive pulse followed by an acquire
    """

    # set up constant pulse for doing a pi pulse
    drive_pulse = SamplePulse(amp * np.ones(total_samples))
    schedule = Schedule()
    schedule |= Play(drive_pulse, DriveChannel(0))
    schedule |= Acquire(total_samples, AcquireChannel(0), MemorySlot(0)) << schedule.duration

    return schedule

def _system_model_1Q(omega_0, r):
    """Constructs a standard model for a 1 qubit system.

    Args:
        omega_0 (float): qubit frequency
        r (float): drive strength

    Returns:
        PulseSystemModel: model for qubit system
    """

    hamiltonian = {}
    hamiltonian['h_str'] = ['2*np.pi*omega0*0.5*Z0', '2*np.pi*r*0.5*X0||D0']
    hamiltonian['vars'] = {'omega0': omega_0, 'r': r}
    hamiltonian['qub'] = {'0': 2}
    ham_model = HamiltonianModel.from_dict(hamiltonian)

    u_channel_lo = []
    subsystem_list = [0]
    dt = 1.

    return PulseSystemModel(hamiltonian=ham_model,
                            u_channel_lo=u_channel_lo,
                            subsystem_list=subsystem_list,
                            dt=dt)

backend_sim = PulseSimulator()

In [3]:
# qubit frequency and drive frequency
omega_0 = 5.
omega_d = omega_0

# drive strength and length of pulse
r = 0.02
total_samples = 100

system_model = _system_model_1Q(omega_0, r)

# set up constant pulse for doing a pi pulse
schedule = _1Q_constant_sched(total_samples)

# set up schedule and qobj
qobj = assemble([schedule],
                backend=backend_sim,
                meas_level=2,
                meas_return='single',
                meas_map=[[0]],
                qubit_lo_freq=[omega_d],
                memory_slots=1,
                shots=1)

# set backend backend_options including initial state
y0 = np.array([1.0, 0.0])
backend_options = {'initial_state' : y0}

# run simulation
%time result = backend_sim.run(qobj, system_model=system_model, backend_options=backend_options).result()

0.1585531234741211
CPU times: user 162 ms, sys: 574 µs, total: 162 ms
Wall time: 163 ms


/Users/dpuzzuoli/Documents/projects/qiskit-terra/qiskit/pulse/library/sample_pulse.py:40: DeprecationWarning: SamplePulse has been renamed to Waveform and is deprecated. Please replace SamplePulse(samples, channel) with Waveform(samples, channel).
  warnings.warn("SamplePulse has been renamed to Waveform and is deprecated. " +


In [4]:
result.results[0].__dict__

{'_metadata': {'seed_simulator': 2097378286,
  'time_taken': 0.15939998626708984},
 'shots': 1,
 'success': True,
 'data': <qiskit.result.models.ExperimentResultData at 0x7f82d8a2b5b0>,
 'meas_level': 2,
 'header': namespace(memory_slots=1, name='sched0', ode_t=100.0),
 'status': 'DONE',
 'meas_return': 'single'}